### In this Notebook we will fetch match level data from data lake folder and insert it into Azure Sql-server RD database.

In [1]:
# !pip install tqdm # if you face issue related to tqdm
import pyodbc
import pandas as pd
from tqdm import tqdm_notebook as tqdm

DATA_LAKE_FOLDER_PATH = "E:/Google_Drive_Contents/data/Final_data_files/"

# fill the database cred here
SERVER = '*'
DATABASE = '*'
USERNAME = '*'
PASSWORD = '*'

In [2]:
# read match_data from data folder

match_df = pd.read_csv(DATA_LAKE_FOLDER_PATH+'/Full_match_level_data_for_DB.csv')

In [3]:
# view some data

match_df.head()

,Ground,Team 1,Team 2,Winner,year,month,day,win_mode,margin,toss_winner,toss_decision
0,Cape Town,Chennai Super Kings,Mumbai Indians,Mumbai Indians,2009,Apr,18,Runs,19.0,Chennai Super Kings,elected to field first
1,Cape Town,Royal Challengers Bangalore,Rajasthan Royals,Royal Challengers Bangalore,2009,Apr,18,Runs,75.0,Royal Challengers Bangalore,elected to bat first
2,Cape Town,Delhi Capitals,Kings XI Punjab,Delhi Capitals,2009,Apr,19,Wickets,10.0,Delhi Capitals,elected to field first
3,Cape Town,Deccan Chargers,Kolkata Knight Riders,Deccan Chargers,2009,Apr,19,Wickets,8.0,Kolkata Knight Riders,elected to bat first
4,Port Elizabeth,Royal Challengers Bangalore,Chennai Super Kings,Chennai Super Kings,2009,Apr,20,Runs,92.0,Chennai Super Kings,elected to bat first


In [4]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764 entries, 0 to 763
Data columns (total 11 columns):
Ground           764 non-null object
Team 1           764 non-null object
Team 2           764 non-null object
Winner           764 non-null object
year             764 non-null int64
month            764 non-null object
day              764 non-null object
win_mode         744 non-null object
margin           744 non-null float64
toss_winner      764 non-null object
toss_decision    764 non-null object
dtypes: float64(1), int64(1), object(9)
memory usage: 65.7+ KB


In [5]:
match_df.isna().sum()

Ground            0
Team 1            0
Team 2            0
Winner            0
year              0
month             0
day               0
win_mode         20
margin           20
toss_winner       0
toss_decision     0
dtype: int64

In [6]:
# treat missing values before inserting to database
match_df['win_mode'] = match_df['win_mode'].fillna('')
match_df['margin'] = match_df['margin'].fillna(0.0)

In [7]:
month_dict = {'Apr':4, 'May':5, 'Jun':6, 'Mar':3}
match_df['month'] = match_df['month'].apply(lambda x : month_dict[x])

In [8]:
def make_date(row):
    return '{}-{}-{}'.format(str(row['year']),str(row['month']),str(row['day']))

match_df['match_date'] = match_df.apply(make_date,axis = 1)

match_df.head()

match_df = match_df[['match_date','Ground', 'Team 1', 'Team 2', 'Winner', \
                     'year', 'month', 'day','win_mode', 'margin', 'toss_winner', 'toss_decision']]

match_df.drop(341,inplace = True)
match_df.head()

,match_date,Ground,Team 1,Team 2,Winner,year,month,day,win_mode,margin,toss_winner,toss_decision
0,2009-4-18,Cape Town,Chennai Super Kings,Mumbai Indians,Mumbai Indians,2009,4,18,Runs,19.0,Chennai Super Kings,elected to field first
1,2009-4-18,Cape Town,Royal Challengers Bangalore,Rajasthan Royals,Royal Challengers Bangalore,2009,4,18,Runs,75.0,Royal Challengers Bangalore,elected to bat first
2,2009-4-19,Cape Town,Delhi Capitals,Kings XI Punjab,Delhi Capitals,2009,4,19,Wickets,10.0,Delhi Capitals,elected to field first
3,2009-4-19,Cape Town,Deccan Chargers,Kolkata Knight Riders,Deccan Chargers,2009,4,19,Wickets,8.0,Kolkata Knight Riders,elected to bat first
4,2009-4-20,Port Elizabeth,Royal Challengers Bangalore,Chennai Super Kings,Chennai Super Kings,2009,4,20,Runs,92.0,Chennai Super Kings,elected to bat first


In [9]:
match_df['match_date'] =  pd.to_datetime(match_df['match_date'], format='%Y/%m/%d')

In [10]:
match_df.to_csv(DATA_LAKE_FOLDER_PATH+'ipl_matches.csv',index = False)

In [121]:
def get_azure_database_connection(server,database,username,password):
    """
    This function will return an aws conn object for azure SQl Server database.
    return : AWS connection object
    """
    cnxn = None
    try:
        cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
        return cnxn
        
    except Exception as e:
        print(e)

In [122]:
# get the database conncetion

db_connection = get_azure_database_connection(SERVER,DATABASE,USERNAME,PASSWORD)

In [123]:
def insert_data_into_database(db_connection,df):
    """
    This function will insert data into the database row by row.
    parameters :
    db_connection : database connection object
    df : pandas DataFrame which has data
    return : None
    """
    try:
        
         # get the cursor
        cur = db_connection.cursor()
        for i,row in tqdm(df.iterrows(),total = df.shape[0]):
            row_list = list(row) # for easy access
            # create insert statment
            Insert_Statment = f"""Insert into ipl_matches (match_date,ground,team_1,team_2,winner,match_year,match_month,match_day,win_mode,margin,toss_winner,toss_decision)values('{row_list[0]}','{row_list[1]}','{row_list[2]}','{row_list[3]}','{row_list[4]}','{row_list[5]}','{row_list[6]}',
                 '{row_list[7]}','{row_list[8]}',{row_list[9]},'{row_list[10]}','{row_list[11]}')"""

#           print(Insert_Statment)

#             break
            # execute the query
            cur.execute(Insert_Statment)
            # commit the connection
            db_connection.commit()


    except Exception as e:
        print(e)

In [124]:
insert_data_into_database(db_connection,match_df)

In [125]:
db_connection.close()

In [126]:
match_df.shape

(763, 12)